In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [2]:
from qml_hep_lhc.data import MNIST
from qml_hep_lhc.layers.utils import convolution_iters
import argparse

2022-08-08 15:36:02.738488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-08 15:36:02.738586: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-08 15:36:56.876235: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-08 15:36:56.876295: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-08-08 15:36:56.906272: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [4]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.labels_to_categorical = 1
args.processed = 1
args.percent_samples = 0.05

# Base Model
args.wandb = False
args.batch_size = 8
args.epochs = 2
args.learning_rate = 0.2


# Quantum CNN Parameters
args.n_layers = 1
args.n_qubits = 1
args.template = 'NQubitPQC'

In [5]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤═══════════════╤══════════════╤═══════════════╤═══════════╕
│ Data   │ Train size    │ Val size     │ Test size     │ Dims      │
╞════════╪═══════════════╪══════════════╪═══════════════╪═══════════╡
│ X      │ (24, 8, 8, 1) │ (6, 8, 8, 1) │ (10, 8, 8, 1) │ (8, 8, 1) │
├────────┼───────────────┼──────────────┼───────────────┼───────────┤
│ y      │ (24, 2)       │ (6, 2)       │ (10, 2)       │ (2,)      │
╘════════╧═══════════════╧══════════════╧═══════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -2.01 │  5.39 │  -0.01 │  0.98 │ [12, 12]

In [8]:
import tensorflow as tf
import numpy as np
import wandb
from tensorflow.keras.callbacks import ReduceLROnPlateau


tf.keras.backend.set_floatx('float64')


In [9]:
callbacks = []

# LR Scheduler callback
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                          factor=0.1,
                                          patience=5,
                                          min_delta=0.0001,
                                          min_lr=1e-6)
callbacks.append(lr_scheduler_callback)

if args.wandb:
    run_id = wandb.util.generate_id()
    wandb.init(project='qml-hep-lhc',
                       config=vars(args),
                       id=run_id,
                       resume='allow')
    callbacks.append(wandb.keras.WandbCallback(save_weights_only=True, save_graph=False))

In [10]:
import pennylane as qml


dev = qml.device("lightning.qubit", wires=args.n_qubits)
qubits = list(range(args.n_qubits))
n_inputs = 9

In [11]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_sparse(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    for l in range(args.n_layers):
        for q in qubits:
            qml.Rot(z[l,q,0], z[l,q,1], z[l,q,2], wires= q)
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))         
    return [qml.expval(qml.PauliZ(i)) for i in range(args.n_qubits)]

In [12]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_basic(inputs, weights, bias):
    inputs = inputs + bias
    qml.AngleEmbedding(inputs, wires=range(args.n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(args.n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(args.n_qubits)]

In [13]:
@qml.qnode(dev, diff_method="adjoint")
def qnode_s2d(inputs, weights, bias):
    z = tf.tensordot(weights, inputs, axes = 1) + bias
    qml.SimplifiedTwoDesign(initial_layer_weights=tf.zeros((args.n_qubits,)), 
                            weights=z, 
                            wires=range(args.n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(args.n_qubits)]

In [14]:
def get_node(template, num_layers, num_qubits, num_inputs):
    if template == 'NQubitPQC':
        return qnode_sparse, {
            "weights": (num_layers, num_qubits, 3, num_inputs),
            "bias": (num_layers, num_qubits, 3)
        }
    elif template == 'Basic':
        assert num_inputs == args.n_qubits
        return qnode_basic, {
            "weights": (num_layers, num_qubits),
            "bias": (num_qubits,)
        }
    elif template == 'S2D':
        return qnode_s2d, {
            "weights": (num_layers, num_qubits -1, 2, num_inputs),
            "bias": (num_layers, num_qubits -1, 2)
        }

In [16]:
inputs = tf.random.uniform((n_inputs, ))
node, shapes = get_node(args.template, args.n_layers, args.n_qubits, n_inputs)
w = tf.random.uniform(shapes['weights'])
b = tf.random.uniform(shapes['bias'])
drawer = qml.draw(node, expansion_strategy="device")
print(drawer(inputs,w,b))

0: ──Rot(2.92,1.70,1.87)─┤  <Z>


In [17]:
node(inputs,w,b)

tensor([-0.12819527], requires_grad=True)

In [176]:
class QConv2D(tf.keras.layers.Layer):
    """
    2D Quantum convolution layer (e.g. spatial convolution over images).
    This layer creates a convolution kernel that is convolved 
    with the layer input to produce a tensor of outputs. Finally,
    `activation` is applied to the outputs as well.
    """

    def __init__(
            self,
            template = 'Basic',
            name='QConv2D',
    ):

        super(QConv2D, self).__init__(name=name)

        self.kernel_size = (3,3)
        self.strides = (2,2)
        
        node, shapes = get_node(template, args.n_layers, args.n_qubits, np.prod(self.kernel_size))
        self.conv_pqc = qml.qnn.KerasLayer(node, shapes, output_dim= args.n_qubits)
        
    def build(self, input_shape):
        
        assert input_shape[3] == 1
    
        self.iters, _ = convolution_iters(
            input_shape[1:3], self.kernel_size, self.strides, 'valid')

    def call(self, x):
        x = tf.image.extract_patches(images=x,
                                       sizes=[1, self.kernel_size[0], self.kernel_size[1], 1],
                                       strides=[1, self.strides[0], self.strides[1], 1],
                                       rates=[1, 1, 1, 1],
                                       padding='VALID')
        x = tf.reshape(x, [-1, np.prod(self.kernel_size)])
        x = tf.map_fn(self.conv_pqc, x)
        x = tf.reshape(x,[-1, self.iters[0],self.iters[1],1])
        return x

In [177]:
rlayer = QConv2D(template = args.template)
flayer = tf.keras.layers.Flatten()
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([rlayer, flayer, clayer_2])

In [178]:
opt = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
model.compile(opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [179]:
dim = data.config()['input_dims']
in_shape = [args.batch_size] + list(dim)

x = tf.random.uniform((in_shape))
y = model(x)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 QConv2D (QConv2D)           multiple                  30        
                                                                 
 flatten_294 (Flatten)       multiple                  0         
                                                                 
 dense_18 (Dense)            multiple                  20        
                                                                 
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


In [175]:
fitting = model.fit(data.train_ds,
                    batch_size= args.batch_size,
                   epochs= args.epochs,
                   validation_data=data.val_ds,
                   shuffle=True,
                    callbacks = callbacks,
                   workers=4)

Epoch 1/2
3/3 [==============================] - 3s 899ms/step - loss: 0.4280 - accuracy: 0.8333 - val_loss: 0.7295 - val_accuracy: 0.6667 - lr: 0.2000
Epoch 2/2
3/3 [==============================] - 3s 956ms/step - loss: 0.6013 - accuracy: 0.7917 - val_loss: 0.6465 - val_accuracy: 0.8333 - lr: 0.2000
